<a href="https://colab.research.google.com/github/gabrielfernandorey/ITBA-NLP/blob/main/Colab%20copia%20NLP_02_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo Practico NLP - Detección de Tópicos y clasificación
- ITBA 2024
- Alumno: Gabriel Rey
---

### NLP_02_model

Esta notebook se utiliza para:
- armar el modelo de estimación de tópicos
- validar resultados obtenidos
- guardar datos en los indices de opensearch

Esta y las consecuentes notebooks son el desarrollo de base de procesos y funciones para la web app provista.

### MODELO

In [7]:
%%capture
from google.colab import userdata

git_token = userdata.get('GIT_TOKEN')
git_username = "gabrielfernandorey"
git_repository = "ITBA-NLP.git"

!git clone https://{git_token}@github.com/{git_username}/{git_repository}

In [142]:
%%capture
!pip install python-dotenv
!pip install bertopic
!pip install OpenAI
!pip install datasets

In [143]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_colwidth', None)
import numpy as np
from matplotlib import pyplot as plt
import os, sys
import json
import pickle
from datetime import datetime, date
from dateutil.parser import parse
from dotenv import load_dotenv

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import NMF

from tqdm import tqdm

from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer

from openai import OpenAI
from datasets import load_dataset

In [28]:
import unicodedata
from functools import wraps
import re
class Cleaning_text:
    '''
    Limpiar elementos no deseados del texto
    '''

    def __init__(self):
        # Definir los caracteres Unicode no deseados
        self.unicode_pattern    = ['\u200e', '\u200f', '\u202a', '\u202b', '\u202c', '\u202d', '\u202e', '\u202f']
        self.urls_pattern       = re.compile(r'http\S+')
        self.simbols_chars      = r"""#&’'"`´“”″()[]*+,-.;:/<=>¿?!¡@\^_{|}~©√≠"""                 # Lista de símbolos a eliminar
        self.simbols_pattern    = re.compile(f"[{re.escape(self.simbols_chars)}]")
        self.escape_pattern     = ['\n', '\t', '\r']

    def _clean_decorator(clean_func):
        @wraps(clean_func)
        def wrapper(self, input_data):
            def clean_string(text):
                return clean_func(self, text)

            if isinstance(input_data, str):
                return clean_string(input_data)
            elif isinstance(input_data, list):
                return [clean_string(item) for item in input_data]
            else:
                raise TypeError("El argumento debe ser una cadena o una lista de cadenas.")
        return wrapper

    @_clean_decorator
    def unicode(self, text):
        for pattern in self.unicode_pattern:
            text = text.replace(pattern, ' ')
        return text

    @_clean_decorator
    def urls(self, text):
        return self.urls_pattern.sub(' ', text)

    @_clean_decorator
    def simbols(self, text):
        return self.simbols_pattern.sub(' ', text)

    @_clean_decorator
    def accents_emojis(self, text):
        return unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')

    @_clean_decorator
    def escape_sequence(self, text):
        for pattern in self.escape_pattern:
            text = text.replace(pattern, ' ').strip()
        return text

    @_clean_decorator
    def str_lower(self, text):
        return text.lower()

#---------------------------------------------------------------------------------------------------------
def clean_all(entities, accents=True, lower=True) -> list:
    """
    Función que toma una lista de entidades, realiza una operación de limpieza
    y devuelve una lista de entidades limpias.
    """
    cleaner = Cleaning_text()

    entities_clean = []
    for ent in entities:
        clean_txt = cleaner.unicode(ent)
        clean_txt = cleaner.urls(clean_txt)
        clean_txt = cleaner.simbols(clean_txt)

        if accents:
            clean_txt = cleaner.accents_emojis(clean_txt)

        clean_txt = cleaner.escape_sequence(clean_txt)

        if lower:
            clean_txt = cleaner.str_lower(clean_txt)

        entities_clean.append(" ".join(clean_txt.split()))

    return entities_clean

### Path

In [ ]:
load_dotenv()
PATH_REMOTO='/content/ITBA-NLP/data/'
PATH=os.environ.get('PATH_LOCAL', PATH_REMOTO)
PATH

'C:/Users/gabri/OneDrive/Machine Learning/Github/ITBA-NLP/data/'

In [23]:
if PATH == os.environ.get('PATH_LOCAL'):
    if os.environ.get('OPENAI_API_KEY'):
        client = OpenAI(api_key= os.environ.get('OPENAI_API_KEY'))
    else:
        client = None
    print(client)

### Data de noticias original

In [163]:
def get_data_news_parquet(date_choice):
    # Desde Hugginface
    path_file = f"jganzabalseenka/news_{date_choice}_24hs"
    dataset = load_dataset(path_file)
    df_parquet = pd.DataFrame(dataset['train'])
    return df_parquet

def valid_data_news(df_parquet, date_choice):
    # Validar datos para la fecha
    choice = "".join(date_choice.split('-'))
    df_parquet.sort_values("start_time_local", ascending=True, inplace=True)
    df_date_filtered = df_parquet[df_parquet['start_time_local'].dt.date == pd.to_datetime(choice).date()]
    print(f"Registros para la fecha {choice} -> {len(df_date_filtered)} de un total de {len(df_parquet)}")
    return df_date_filtered

def get_batch_news(df_date_filtered, batch_news):
    # Obtenemos un lote de N noticias por dia (para agilizar el procesamiento)
    df_batch = df_date_filtered.sample(n=int(batch_news)).reset_index()
    data_1 = list(df_batch['text'])
    id_data_1 = list(df_batch['asset_id'])
    title_data_1 = list(df_batch['title'])
    print(f"Batch: {len(data_1)}")
    return data_1, id_data_1, title_data_1

def clean_data_news(data):
    clean_data = Cleaning_text()
    proc_data = []
    for data_in in tqdm(data):
        aux = clean_data.unicode(data_in)
        aux = clean_data.urls(aux)
        aux = clean_data.simbols(aux)
        aux = clean_data.escape_sequence(aux)
        aux = " ".join([ word for word in aux.split() if word.lower() not in SPANISH_STOPWORDS_SPECIAL])
        proc_data.append(aux)
    return proc_data

### Preprocesar las noticias
Se realiza un preprocesamiento mínimo del texto, pero no se le quita el sentido semántico para que mediante SentenceTransformer se puedan capturar embeddings de mejor calidad.

In [25]:
# Stopwords
SPANISH_STOPWORDS = list(pd.read_csv(PATH+'spanish_stop_words.csv' )['stopwords'].values)
SPANISH_STOPWORDS_SPECIAL = list(pd.read_csv(PATH+'spanish_stop_words_spec.csv' )['stopwords'].values)

### Modelo

In [162]:
#### OnLine Topic Modeling
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import IncrementalPCA
from bertopic.vectorizers import OnlineCountVectorizer

# Step 1 - Extract embeddings
embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Prepare sub-models that support online learning
umap_model = IncrementalPCA(n_components=5)
cluster_model = MiniBatchKMeans(n_clusters=50, random_state=42)
vectorizer_model = OnlineCountVectorizer(stop_words=SPANISH_STOPWORDS, decay=.01)

# Step 5 - Create topic representation
ctfidf_model = ClassTfidfTransformer()

topic_online_model = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=cluster_model,
    vectorizer_model=vectorizer_model,
    ctfidf_model=ctfidf_model,
    verbose=True
    )

### Entrenamiento

In [164]:
dates_list = ['2024-07-16', '2024-07-17']
batch_news = 1000
titles = []
topics = []
for date_choice in dates_list:
    df_parquet = get_data_news_parquet(date_choice)
    df_date_filtered = valid_data_news(df_parquet, date_choice)
    data, id_data, title_data = get_batch_news(df_date_filtered, batch_news)

    titles += title_data
    proc_data = clean_data_news(data)
    topics_online = topic_online_model.partial_fit(proc_data)
    topics.extend(topic_online_model.topics_)



Registros para la fecha 20240716 -> 13687 de un total de 17283
Batch: 1000


100%|██████████| 1000/1000 [00:02<00:00, 474.64it/s]


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

2024-08-25 00:55:53,842 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-08-25 00:55:54,044 - BERTopic - Dimensionality - Completed ✓
2024-08-25 00:55:54,051 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-08-25 00:55:54,073 - BERTopic - Cluster - Completed ✓


Registros para la fecha 20240717 -> 11663 de un total de 16037
Batch: 1000


100%|██████████| 1000/1000 [00:02<00:00, 476.24it/s]


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

2024-08-25 00:58:37,021 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-08-25 00:58:37,326 - BERTopic - Dimensionality - Completed ✓
2024-08-25 00:58:38,179 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-08-25 00:58:38,187 - BERTopic - Cluster - Completed ✓


In [182]:
topic_id = np.random.randint(len(set(topic_online_model.topics_))-1) # Obtener un topico al azar
topic_keywords = topic_online_model.get_topics()[topic_id]
docs_position_per_topic = [i for i, x in enumerate(topics) if x == topic_id]
print(topic_id, "_".join([keyword[0] for keyword in topic_keywords[:4]]))
for doc in docs_position_per_topic:
    print(titles[doc])

37 mercado_dólares_dólar_pesos
Adorni respaldó las medidas de Caputo: «No nos ocupa el dólar o el riesgo país, estamos obsesionados con el peso"
Confirmaron las fechas de las audiencias para tratar los pliegos de Lijo y García Mansilla en el Senado
Ampros rechazó la última oferta del gobierno de Cornejo y seguirán las medidas de fuerza
En el campo no ven motivos para desprenderse de la soja y crece la preocupación en el Gobierno
Exdirector del Banco Central advirtió sobre los riesgos del nuevo plan monetario: "Lo que están haciendo no le va a gustar al Fondo"
ANSES continúa el calendario de pagos de AUH y otras Asignaciones Familiares: quiénes cobran hoy, martes 16 de julio de 2024
El primer round quedó para el Gobierno de Milei: el dólar cayó a $ 1.415
"El mercado ve dudas con el dólar, no aprecia hacia dónde quiere ir el gobierno nacional".
Multas para ART: Gobierno fijó nuevo monto a partir de julio
Fintech argentina recaudó $22 millones de Kaszek y Andreessen Horowitz
Dólar hoy: vu

In [189]:
embeddings = topic_online_model.embedding_model.embed(data)

In [190]:
def calculate_probabilities(model, docs):
    # Transformar documentos al espacio vectorial
    embeddings = model.embedding_model.embed(docs)

    # Aplicar reducción de dimensionalidad
    reduced_embeddings = model.umap_model.transform(embeddings)

    # Predecir clústeres y calcular distancias
    clusters = model.hdbscan_model.predict(reduced_embeddings)
    distances = model.hdbscan_model.transform(reduced_embeddings)

    # Aproximar probabilidades (probabilidad inversamente proporcional a la distancia)
    max_distances = np.max(distances, axis=1, keepdims=True)
    probs = 1 - (distances / max_distances)
    return clusters, probs

In [191]:
# Obtener probabilidades para un nuevo conjunto de datos
topics_x, probs_x = calculate_probabilities(topic_online_model, data)

In [192]:
def topic_data_view(model, df, probs=None):
    reg = []
    for topic_number in range(len(model.get_topic_info())):
        docs_per_topics = [i for i, x in enumerate(model.topics_) if x == topic_number]
        for doc in docs_per_topics:
            if probs is None:
                reg.append([str(topic_number), str(doc), df.iloc[doc].title])
                columns=['Topic','Id Doc','Title']
            else:
                prob = np.max(probs[doc])
                reg.append([str(topic_number), str(doc), df.iloc[doc].title, prob])
                columns=['Topic','Id Doc','Title','Probs']

    df_query = pd.DataFrame(reg, columns=columns)
    return df_query

In [193]:
topic_data_view(topic_online_model, df_parquet, probs_x)

,Topic,Id Doc,Title,Probs
0,0,31,"La lomense Helen Bernard Stilling, campeona sudamericana U20 en atletismo",0.787337
1,0,99,La drástica decisión del Chelsea con Enzo Fernández tras su canto racista,0.708612
2,0,109,Vacantes en la Corte Suprema: ya hay fecha para tratar los pliegos de Lijo y García Mansilla en el Senado,0.669151
3,0,121,Desmantelaron en San Juan una red de trata que trasladaba mujeres a Colombia | Hay tres detenidos,0.776490
4,0,152,Dalma Maradona explotó contra Eduardo Feinmann: el motivo del enojo,0.704144
...,...,...,...,...
995,49,674,Incendio en la escuela de Caviahue: el jueves finalizaría la adecuación del hotel del ISSN,0.780474
996,49,778,La Plata: las dos medidas impulsadas por la municipalidad tras el macabro hallazgo de restos abandonados en el cementerio,0.760354
997,49,903,"Pepe Cibrián apuntó contra el hijo de Adrián Suar y aseguró que demandará al streaming Olga: ""Asco""",0.792598
998,49,914,Hoy martes 16 de Julio a las 22 hs. cierra la encuesta.,0.742581
